### Import required libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, DistilBertTokenizerFast
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
import time

### Example Sentences

In [12]:
tricky_sentences = [
...    "Most automated sentiment analysis tools are shit.",
...    "VADER sentiment analysis is amazingly bad.",
...    "Sentiment analysis has never been good.",
...    "Sentiment analysis with VADER has never been this good.",
...    "Warren Beatty has never been so entertaining.",
...    "I won't say that the movie is astounding and I wouldn't claim that \
...    the movie is too banal either.",
...    "I like to hate Michael Bay films, but I couldn't fault this one",
...    "I like to hate Michael Bay films, BUT I couldn't help but fault this one",
...    "It's one thing to watch an Uwe Boll film, but another thing entirely \
...    to pay for it",
...    "The movie was too good",
...    "This movie was actually neither that funny, nor super witty.",
...    "This movie doesn't care about cleverness, wit or any other kind of \
...    intelligent humor.",
...    "Those who find ugly meanings in beautiful things are corrupt without \
...    being charming.",
...    "There are slow and repetitive parts, BUT it has just enough spice to \
...    keep it interesting.",
...    "The script is not fantastic, but the acting is decent and the cinematography \
...    is EXCELLENT!",
...    "Roger Dodger is one of the most compelling variations on this theme.",
...    "Roger Dodger is one of the least compelling variations on this theme.",
...    "Roger Dodger is at least compelling as a variation on the theme.",
...    "they fall in love with the product",
...    "but then it breaks",
...    "usually around the time the 90 day warranty expires",
...    "the twin towers collapsed today",
...    "However, Mr. Carter solemnly argues, his client carried out the kidnapping \
...    under orders and in the ''least offensive way possible.''"
... ]

# My Implementation of DistilBert

In [3]:
tokenizer_1 = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model_1 = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [4]:
def sentiment_score_1(review):
    tokens = tokenizer_1.encode(review, return_tensors='pt')
    result = model_1(tokens)
    prediction = int(torch.argmax(result.logits))

    print(result)
    print(prediction)
    # Classify sentiment
    if prediction == 1:
        sentiment_label = "Positive"
    else:
        sentiment_label = "Negative"

    return sentiment_label # return logits as well?

In [ ]:
for sentence in tricky_sentences:

    sentiment_label = sentiment_score_1(sentence)

    print("Sentence:", sentence)
    print("Sentiment:", sentiment_label)
    print()

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.6491, -3.7305]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
0
Sentence: Most automated sentiment analysis tools are shit.
Sentiment: Negative

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.7243, -3.7802]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
0
Sentence: VADER sentiment analysis is amazingly bad.
Sentiment: Negative

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.6214, -3.6775]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
0
Sentence: Sentiment analysis has never been good.
Sentiment: Negative

SequenceClassifierOutput(loss=None, logits=tensor([[ 1.1775, -0.8812]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
0
Sentence: Sentiment analysis with VADER has never been this good.
Sentiment: Negative

SequenceClassifierOutput(loss=None, logits=tensor([[-3.9581,  4.1617]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
sentiment_score_1("hello, what a bad morning!")


SequenceClassifierOutput(loss=None, logits=tensor([[ 3.1201, -2.5089]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
0


'Negative'

# Small Sample Finetuning of Distilbert
**the code below is just my efforts for a working code, and a check for presence of accuracy metrics' increases**

### Establishing a baseline for comparison

In [ ]:
# Load the model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
model = model.to('cuda')  # if GPU is available

# Load validation data
val_data = pd.read_csv('/content/drive/MyDrive/CZ4034/kaggle_validate_100.csv')
val_texts = val_data['Review'].tolist()
val_labels = val_data['Rating'].tolist()

# Initialize tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize data
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Create torch dataset for validation
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

val_dataset = ReviewDataset(val_encodings, val_labels)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

# Predict with the model
model.eval()
predictions = []
true_labels = []
for batch in val_loader:
    input_ids = batch['input_ids'].to('cuda')
    attention_mask = batch['attention_mask'].to('cuda')
    labels = batch['labels'].to('cuda')

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1).cpu().numpy()
    predictions.extend(predicted_labels)
    true_labels.extend(labels.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)
conf_matrix = confusion_matrix(true_labels, predictions)

print(f'Accuracy: {accuracy}')
print(f'F1-score: {f1}')
print(f'Confusion matrix:\n {conf_matrix}')

Accuracy: 0.5
F1-score: 0.0
Confusion matrix:
 [[50  0]
 [50  0]]


### Actual finetuning process

In [ ]:
# Record start time
start_time = time.time()

# Load data
data = pd.read_csv('/content/drive/MyDrive/CZ4034/kaggle_TRAIN_1000.csv')
reviews = data['Review'].tolist()
labels = data['Rating'].tolist()  # assuming sentiment is encoded as 0 (negative) and 1 (positive)

# Split data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(reviews, labels, test_size=0.2)

# Initialize tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Create torch dataset
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataloaders
train_dataset = ReviewDataset(train_encodings, train_labels)
val_dataset = ReviewDataset(val_encodings, val_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Initialize model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
model = model.to('cuda')  # if GPU is available

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(3):  # number of epochs
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['labels'].to('cuda')
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the model
model.save_pretrained('sentiment_model_DistilBERT')

# Record end time
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Time required to fine-tune:  37.64240622520447


### **Time to see how our fine-tuned model does on the unseen validation dataset.**

In [ ]:
# Load the model
model = DistilBertForSequenceClassification.from_pretrained('sentiment_model_DistilBERT')
model = model.to('cuda')  # if GPU is available

# Load validation data
val_data = pd.read_csv('/content/drive/MyDrive/CZ4034/kaggle_validate_100.csv')
val_texts = val_data['Review'].tolist()
val_labels = val_data['Rating'].tolist()

# Initialize tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Tokenize data
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Create torch dataset for validation
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

val_dataset = ReviewDataset(val_encodings, val_labels)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


In [ ]:

# Evaluate the model
model.eval()
predictions = []
true_labels = []
for batch in val_loader:
    input_ids = batch['input_ids'].to('cuda')
    attention_mask = batch['attention_mask'].to('cuda')
    labels = batch['labels'].to('cuda')

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1).cpu().numpy()
    predictions.extend(predicted_labels)
    true_labels.extend(labels.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)
conf_matrix = confusion_matrix(true_labels, predictions)

print(f'Accuracy: {accuracy}')
print(f'F1-score: {f1}')
print(f'Confusion matrix:\n {conf_matrix}')

Accuracy: 0.99
F1-score: 0.98989898989899
Confusion matrix:
 [[50  0]
 [ 1 49]]


# Actual Finetuning of DistilBert with project datasets
**The code below uses our project's full-sized dataset, and external training datasets.**

### Establishing a baseline for comparison

In [5]:
# Load the model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english', num_labels=2)
model = model.to('cuda')  # if GPU is available

# Load validation data
val_data = pd.read_csv('/content/drive/MyDrive/CZ4034/kaggle_validate_balanced_7000.csv')
val_texts = val_data['Review'].tolist()
val_labels = val_data['Rating'].tolist()

# Initialize tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')

# Tokenize data
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Create torch dataset for validation
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

val_dataset = ReviewDataset(val_encodings, val_labels)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


In [6]:

# Predict with the model
model.eval()
predictions = []
true_labels = []
for batch in val_loader:
    input_ids = batch['input_ids'].to('cuda')
    attention_mask = batch['attention_mask'].to('cuda')
    labels = batch['labels'].to('cuda')

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1).cpu().numpy()
    predictions.extend(predicted_labels)
    true_labels.extend(labels.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)
conf_matrix = confusion_matrix(true_labels, predictions)

print(f'Accuracy: {accuracy}')
print(f'F1-score: {f1}')
print(f'Confusion matrix:\n {conf_matrix}')

Accuracy: 0.9308571428571428
F1-score: 0.9266221952698606
Confusion matrix:
 [[3460   40]
 [ 444 3056]]


### Actual finetuning process

In [7]:
# Record start time
start_time = time.time()

# Load data
data = pd.read_csv('/content/drive/MyDrive/CZ4034/kaggle_TRAIN_balanced_30000.csv')
reviews = data['Review'].tolist()
labels = data['Rating'].tolist()  # assuming sentiment is encoded as 0 (negative) and 1 (positive)

# Split data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(reviews, labels, test_size=0.2)

# Initialize tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')

# Tokenize data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Create torch dataset
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataloaders
train_dataset = ReviewDataset(train_encodings, train_labels)
val_dataset = ReviewDataset(val_encodings, val_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Initialize model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english', num_labels=2)
model = model.to('cuda')  # if GPU is available

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(3):  # number of epochs
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['labels'].to('cuda')
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the model
model.save_pretrained('sentiment_model_DistilBERT_CZ4034')

# Record end time
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)

Time required to fine-tune:  1063.8676331043243


### **Time to see how our fine-tuned model does on the unseen validation dataset.**

In [8]:
# Load the model
model = DistilBertForSequenceClassification.from_pretrained('sentiment_model_DistilBERT_CZ4034')
model = model.to('cuda')  # if GPU is available

# Load validation data
val_data = pd.read_csv('/content/drive/MyDrive/CZ4034/kaggle_validate_balanced_7000.csv')
val_texts = val_data['Review'].tolist()
val_labels = val_data['Rating'].tolist()

# Initialize tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')

# Tokenize data
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Create torch dataset for validation
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

val_dataset = ReviewDataset(val_encodings, val_labels)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Evaluate the model
model.eval()
predictions = []
true_labels = []
for batch in val_loader:
    input_ids = batch['input_ids'].to('cuda')
    attention_mask = batch['attention_mask'].to('cuda')
    labels = batch['labels'].to('cuda')

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1).cpu().numpy()
    predictions.extend(predicted_labels)
    true_labels.extend(labels.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)
conf_matrix = confusion_matrix(true_labels, predictions)

print(f'Accuracy: {accuracy}')
print(f'F1-score: {f1}')
print(f'Confusion matrix:\n {conf_matrix}')

Accuracy: 0.9882857142857143
F1-score: 0.9882588774341351
Confusion matrix:
 [[3467   33]
 [  49 3451]]


## **Executing on final cleaned dataset**

In [9]:
tokenizer_finetuned = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model_finetuned = DistilBertForSequenceClassification.from_pretrained("sentiment_model_DistilBERT_CZ4034")


In [10]:
def sentiment_score_finetuned(review):
    tokens = tokenizer_finetuned.encode(review, return_tensors='pt')
    result = model_finetuned(tokens)
    prediction = int(torch.argmax(result.logits))

    # Classify sentiment
    if prediction == 1:
        sentiment_label = "Positive"
    else:
        sentiment_label = "Negative"

    return sentiment_label # return logits as well?

In [16]:
for sentence in tricky_sentences:

    sentiment_label = sentiment_score_finetuned(sentence)

    #print("Sentence:", sentence)
    print("Sentiment:", sentiment_label)
    print()

Sentiment: Negative

Sentiment: Negative

Sentiment: Negative

Sentiment: Positive

Sentiment: Positive

Sentiment: Negative

Sentiment: Negative

Sentiment: Negative

Sentiment: Negative

Sentiment: Positive

Sentiment: Negative

Sentiment: Negative

Sentiment: Negative

Sentiment: Positive

Sentiment: Positive

Sentiment: Positive

Sentiment: Positive

Sentiment: Positive

Sentiment: Positive

Sentiment: Negative

Sentiment: Positive

Sentiment: Negative

Sentiment: Negative



In [17]:
# Import dataset, using first column as index column
glassdoor_data_distilbert_finetuned = pd.read_csv('/content/drive/MyDrive/CZ4034/final_sg_companies_reviews_clean.csv')

# Concatenating Pros and Cons
glassdoor_data_distilbert_finetuned['Overall Review'] = glassdoor_data_distilbert_finetuned['Pros'] + '. ' + glassdoor_data_distilbert_finetuned['Cons']

In [18]:
glassdoor_data_distilbert_finetuned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18349 entries, 0 to 18348
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Company Name    18349 non-null  object 
 1   Overall Rating  18349 non-null  float64
 2   Review Date     18349 non-null  object 
 3   Review Title    18349 non-null  object 
 4   Job Title       18342 non-null  object 
 5   Job Details     18349 non-null  object 
 6   Location        13005 non-null  object 
 7   Pros            18349 non-null  object 
 8   Cons            18349 non-null  object 
 9   Overall Review  18349 non-null  object 
dtypes: float64(1), object(9)
memory usage: 1.4+ MB


In [19]:
#glassdoor_data_distilbert_finetuned["Pro's Sentiment"] = glassdoor_data_distilbert_finetuned['Pros'].apply(lambda x: sentiment_score_finetuned(x[:512]))
#glassdoor_data_distilbert_finetuned["Cons's Sentiment"] = glassdoor_data_distilbert_finetuned['Cons'].apply(lambda x: sentiment_score_finetuned(x[:512]))
glassdoor_data_distilbert_finetuned['Predicted Sentiment'] = glassdoor_data_distilbert_finetuned['Overall Review'].apply(lambda x: sentiment_score_finetuned(x[:512]))
glassdoor_data_distilbert_finetuned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18349 entries, 0 to 18348
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Company Name         18349 non-null  object 
 1   Overall Rating       18349 non-null  float64
 2   Review Date          18349 non-null  object 
 3   Review Title         18349 non-null  object 
 4   Job Title            18342 non-null  object 
 5   Job Details          18349 non-null  object 
 6   Location             13005 non-null  object 
 7   Pros                 18349 non-null  object 
 8   Cons                 18349 non-null  object 
 9   Overall Review       18349 non-null  object 
 10  Predicted Sentiment  18349 non-null  object 
dtypes: float64(1), object(10)
memory usage: 1.5+ MB


In [20]:
# Export DataFrame to CSV
glassdoor_data_distilbert_finetuned.to_csv('/content/drive/MyDrive/CZ4034/glassdoor_data_distilbert_finetuned.csv', index=False)